# From files to separated chunk documents... in a single step

In Notebook #1 we learnt how to create an index in Azure AI Search and leverage its "pull" ability to extract data from files located in storage account. As we saw, this "extract and fill" tecnique can be integrated by a "skillset" that "enriches" extracted data before writing them into the index. Such process is orchestrated by a so called "indexer" that maps each source or enriched piece of information against the proper field of the target index.<br/>
If we need to split a long text field -typically, the *content* field- in chunks no longer than a predefined length -e.g. 5,000 characters-, we can leverage the **SplitSkill** skill of the skillset, which generates a multi-value field -*Collection(Edm.String)* type- containing the different chunks.<br/>
This means, on the other hand, that if we need to retrieve the best ***chunks*** rather than the ***whole content*** of documents due to OpenAI token limits, or just because the whole document contains too much and potentially more confusing information, then we need to create a **secondary index** where we store **one chunk per index row**, possibly associating  specific embeddings, key phrases, emails/urls/persons and other entities to each search document (=chunk).<br/>
That's exactly what we did in our Notebook #3; however, in this case we had to face with a few main challenges:
- Filling the second index requires a set of manual steps, which we easily wrote in Python, but which creates a potential bottleneck when our scenario needs maintenance and scalability. For example, we had to manually vectorize each chunk and manually *push* the new search document into the secondary index
- At the end we get two indexes to manage, where the first one becomes potentially useless
- We need to define rules to keep both indexes in sync, and identify update policies. For example, in Notebook #3 we decided to *project and vectorize* only the chunks extracted by a first query, leaving the remaning chunks in their arrays within the first index.

These were design decision led by some practical considerations and the general skill-up objective of this solution accelerator; however, they were also forced by some limits associated to Azure AI Search, *until a few days ago*.<br/>
Yes, because with the release of the [latest stable version 2023-11-01 of Azure AI Search REST API](https://learn.microsoft.com/en-us/rest/api/searchservice/search-service-api-versions#stable-versions), we can now operate all the above actions with the following benefits:
- **pull mathod**, e.g. no manual steps to upload search document to target index
- **automatic embedding creation**, that can be performed by a specific skill
- **automatic projection** of the array values into separated search documents of the target index

As a matter of fact, we actually do not need two indexes any more. **Let's see how to do it!**

In [1]:
# Load libraries and assign variables

import os
import json
import requests
from dotenv import load_dotenv
load_dotenv("credentials.env")

# Name of the container in your Blob Storage Datasource ( in credentials.env)
BLOB_CONTAINER_NAME = "arxivcs"

In [2]:
# Define the names for the data source, skillset, index and indexer
datasource_name = "cogsrch-datasource-files-onestep"
skillset_name   = "cogsrch-skillset-files-onestep"
index_name      = "cogsrch-index-files-vector-onestep"
indexer_name    = "cogsrch-indexer-files-onestep"

In [3]:
# Setup the Payloads header
headers     = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params      = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}
params_old  = {'api-version': os.environ['AZURE_SEARCH_API_VERSION_OLD']} # needed for skillset creation with projection

## Create Data Source (Blob container with the Arxiv CS pdfs)

In [4]:
# The following code sends the json paylod to Azure Search engine to create the Datasource

datasource_payload = {
    "name": datasource_name,
    "description": "Demo files to demonstrate cognitive search capabilities.",
    "type": "azureblob",
    "credentials": {
        "connectionString": os.environ['BLOB_CONNECTION_STRING']
    },
    "dataDeletionDetectionPolicy" : {
        "@odata.type" :"#Microsoft.Azure.Search.NativeBlobSoftDeleteDeletionDetectionPolicy" # this makes sure that if the item is deleted from the source, it will be deleted from the index
    },
    "container": {
        "name": BLOB_CONTAINER_NAME
    }
}
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


## Create the *SINGLE* Index
Since a [vectorizer](https://learn.microsoft.com/en-us/rest/api/searchservice/indexes/create?view=rest-searchservice-2023-10-01-preview&tabs=HTTP#vectorsearch) is used here, we need to use API 2023-10-01-Preview

In [5]:
# Create the single index
index_payload = {
    "name": index_name,
    "fields": [        
        {
          "name": "id",
          "type": "Edm.String",
          "key": "true",
          "searchable": "true",
          "retrievable": "true",
          "sortable": "false",
          "filterable": "false",
          "facetable": "false",
          "analyzer": "keyword"
        },
        {
          "name": "ParentKey",
          "type": "Edm.String",
          "searchable": "true",
          "retrievable": "true",
          "facetable": "false",
          "filterable": "true",
          "sortable": "false"
        },
        {
          "name": "title",
          "type": "Edm.String",
          "searchable": "true",
          "retrievable": "true"
        },
        {
          "name": "chunk",
          "type": "Edm.String",
          "searchable": "true",
          "retrievable": "true"
        },
        {
          "name": "name",
          "type": "Edm.String",
          "searchable": "true",
          "retrievable": "true",
          "sortable": "false",
          "filterable": "false",
          "facetable": "false"
        },
        {
          "name": "location",
          "type": "Edm.String",
          "searchable": "false",
          "retrievable": "true",
          "sortable": "false",
          "filterable": "false",
          "facetable": "false"
        },
        {
          "name": "language",
          "type": "Edm.String",
          "searchable": "true",
          "retrievable": "true",
          "sortable": "true",
          "filterable": "true",
          "facetable": "true"
        },
        {
          "name": "persons",
          "type": "Collection(Edm.String)",
          "searchable": "true",
          "retrievable": "true",
          "sortable": "false",
          "filterable": "false",
          "facetable": "false"
        },
        {
          "name": "urls",
          "type": "Collection(Edm.String)",
          "searchable": "true",
          "retrievable": "true",
          "sortable": "false",
          "filterable": "false",
          "facetable": "false"
        },
        {
          "name": "emails",
          "type": "Collection(Edm.String)",
          "searchable": "true",
          "retrievable": "true",
          "sortable": "false",
          "filterable": "false",
          "facetable": "false"
        },
        {
          "name": "key_phrases",
          "type": "Collection(Edm.String)",
          "searchable": "true",
          "retrievable": "true",
          "sortable": "false",
          "filterable": "false",
          "facetable": "false"
        },
        {
          "name": "chunkVector",
          "type": "Collection(Edm.Single)",
          "searchable": "true",
          "retrievable": "true",
          "dimensions": 1536,
          "vectorSearchProfile": "my_vectorSearch_profile"
        }
    ],    
  
    "vectorSearch": {        
        "profiles": [
            {                
                "name": "my_vectorSearch_profile",
                "algorithm": "my-vectorSearch-algorithm",
                "vectorizer": "my-embeddings-vectorizer"
            }
        ],        
        "algorithms": [            
            {
                "name": "my-vectorSearch-algorithm",
                "kind": "hnsw",
                "hnswParameters": {                    
                    "m": 4,
                    "metric": "cosine",
                    "efConstruction": 400,
                    "efSearch": 500
                }          
            }
        ],        
        "vectorizers": [
            {
                "name": "my-embeddings-vectorizer",
                "kind": "azureOpenAI",
                "azureOpenAIParameters": {
                "resourceUri": os.environ['AZURE_OPENAI_ENDPOINT'],
                "apiKey": os.environ['AZURE_OPENAI_API_KEY'],
                "deploymentId": os.environ['EMBEDDING_DEPLOYMENT']
                }
            }
        ]        
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {
                        "fieldName": "title"
                    },
                    "prioritizedContentFields": [
                        {
                            "fieldName": "chunk"
                        }
                    ],
                    "prioritizedKeywordsFields": []
                }
            }
        ]
    }
}


r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params_old)
print(r.status_code)
print(r.ok)

201
True


## Create the Skillset
### Please note, in this case we need to use version 2023-10-01-Preview of Azure AI Search REST API which currently offers the projection feature
- [Reference guide version 2023-10-01-Preview](https://learn.microsoft.com/en-us/rest/api/searchservice/skillsets/create-or-update?view=rest-searchservice-2023-10-01-Preview&tabs=HTTP)
- [Reference guide latest version (2023-11-01)](https://learn.microsoft.com/en-us/rest/api/searchservice/skillsets/create-or-update?view=rest-searchservice-2023-11-01&tabs=HTTP)

In [6]:
# Create the skillset, including the projection settings into the single index created above
skillset_payload = {    
  "name": skillset_name,
  "description": "Detect language, run OCR, merge content+ocr_text, split in chunks, extract entities and key-phrases and embed the chunks",
  "skills": [
    {
      "@odata.type": "#Microsoft.Skills.Text.LanguageDetectionSkill",
      "name": "LanguageDetectionSkill",
      "context": "/document",
      "description": "If you have multilingual content, adding a language code is useful for filtering",
      "inputs": [
        {
          "name": "text",
          "source": "/document/content"
        }
      ],
      "outputs": [
        {
          "name": "languageCode",
          "targetName": "language"
        }
      ]
    },
    {
      "@odata.type": "#Microsoft.Skills.Vision.OcrSkill",
      "name": "OcrSkill",
      "description": "",
      "context": "/document/normalized_images/*",
      "textExtractionAlgorithm": "",
      "lineEnding": "Space",
      "defaultLanguageCode": "en",
      "detectOrientation": "true",
      "inputs": [
        {
          "name": "image",
          "source": "/document/normalized_images/*"
        }
      ],
      "outputs": [
        {
          "name": "text",
          "targetName": "text_from_ocr"
        }
      ]
    },
    {
      "@odata.type": "#Microsoft.Skills.Text.MergeSkill",
      "name": "MergeSkill",
      "description": "",
      "context": "/document",
      "insertPreTag": " ",
      "insertPostTag": " ",
      "inputs": [
        {
          "name": "text",
          "source": "/document/content"
        },
        {
          "name": "itemsToInsert",
          "source": "/document/normalized_images/*/text_from_ocr"
        },
        {
          "name": "offsets",
          "source": "/document/normalized_images/*/contentOffset"
        }
      ],
      "outputs": [
        {
          "name": "mergedText",
          "targetName": "merged_text"
        }
      ]
    },
    {
      "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
      "name": "SplitSkill",
      "context": "/document",
      "textSplitMode": "pages",
      "maximumPageLength": 5000,
      "defaultLanguageCode": "en",
      "inputs": [
        {
          "name": "text",
          "source": "/document/merged_text"
        },
        {
          "name": "languageCode",
          "source": "/document/language"
        }
      ],
      "outputs": [
        {
          "name": "textItems",
          "targetName": "chunks"
        }        
      ]
    },
    {
      "@odata.type": "#Microsoft.Skills.Text.LanguageDetectionSkill",
      "name": "LanguageDetectionSkill_by_chunk",
      "context": "/document/chunks/*",
      "description": "If you have multilingual content, adding a language code is useful for filtering",
      "inputs": [
        {
          "name": "text",
          "source": "/document/chunks/*"
        }
      ],
      "outputs": [
        {
          "name": "languageCode",
          "targetName": "chunk_language"
        }
      ]
    },
    {
      "@odata.type": "#Microsoft.Skills.Text.V3.EntityRecognitionSkill",
      "name": "EntityRecognitionSkill",
      "description": "",
      "context": "/document/chunks/*",
      "defaultLanguageCode": "en",
      "minimumPrecision": 0.5,
      "modelVersion": "",
      "inputs": [
        {
          "name": "text",
          "source": "/document/chunks/*"
        },
        {
            "name": "languageCode", 
            "source": "/document/chunks/*/chunk_language"
        }
      ],
      "outputs": [
        {
          "name": "persons",
          "targetName": "persons"
        },
        {
          "name": "urls",
          "targetName": "urls"
        },
        {
          "name": "emails",
          "targetName": "emails"
        }
      ]
    },
    {
      "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",
      "defaultLanguageCode": "en",
      "modelVersion": "",
      "name": "KeyPhraseExtractionSkill",
      "description": "",
      "context": "/document/chunks/*",
      "inputs": [
        {
          "name": "text",
          "source": "/document/chunks/*"
        },
        {
          "name": "languageCode",
          "source": "/document/chunks/*/chunk_language"
        }
      ],
      "outputs": [
        {
          "name": "keyPhrases",
          "targetName": "key_phrases"
        }
      ]
    },
    {
      "@odata.type": "#Microsoft.Skills.Text.AzureOpenAIEmbeddingSkill",
      "name": "AzureOpenAIEmbeddingSkill",
      "description": "",
      "context": "/document/chunks/*",
      "resourceUri": "https://mmopenai04.openai.azure.com",
      "apiKey": "23b1db9a7f3a4eeb8a4f1c74bdd7d13d",
      "deploymentId": "text-embedding-ada-002",
      "inputs": [
        {
          "name": "text",
          "source": "/document/chunks/*"
        }
      ],
      "outputs": [
        {
          "name": "embedding",
          "targetName": "chunk_embedded"
        }
      ]
    }
  ],
  "cognitiveServices": {
    "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
    "description": os.environ['COG_SERVICES_NAME'],
    "key": os.environ['COG_SERVICES_KEY']
  },
  "indexProjections": {
    "selectors": [
      {
        "targetIndexName": index_name,
        "parentKeyFieldName": "ParentKey",
        "sourceContext": "/document/chunks/*",
        "mappings": [
          {
            "name": "title",
            "source": "/document/title"
          },
          {
            "name": "name",
            "source": "/document/name"
          },
          {
            "name": "location",
            "source": "/document/location"
          },
          {
            "name": "chunk",
            "source": "/document/chunks/*"
          },
          {
            "name": "chunkVector",
            "source": "/document/chunks/*/chunk_embedded"
          },
          {
            "name": "language",
            "source": "/document/chunks/*/chunk_language"
          },
          {
            "name": "persons",
            "source": "/document/chunks/*/persons"
          },
          {
            "name": "urls",
            "source": "/document/chunks/*/urls"
          },
          {
            "name": "emails",
            "source": "/document/chunks/*/emails"
          },
          {
            "name": "key_phrases",
            "source": "/document/chunks/*/key_phrases"
          }
        ]
      }
    ]
  }
}


r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params_old)
print(r.status_code)
print(r.ok)

201
True


## Create and Run the Indexer
[Reference guide](https://learn.microsoft.com/en-us/rest/api/searchservice/indexers?view=rest-searchservice-2023-11-01)

In [7]:
# Create the skillset, including the projection settings into the single index created above
indexer_payload = {    
  "name": indexer_name,
  "dataSourceName": datasource_name,
  "targetIndexName": index_name,
  "skillsetName": skillset_name,  
  "fieldMappings": [
    {
      "sourceFieldName": "metadata_storage_path",
      "targetFieldName": "id",
      "mappingFunction": {
        "name": "base64Encode"
      }
    },
    {
      "sourceFieldName": "metadata_title",
      "targetFieldName": "title"
    },
    {
      "sourceFieldName": "metadata_storage_name",
      "targetFieldName": "name"
    },
    {
      "sourceFieldName": "metadata_storage_path",
      "targetFieldName": "location"
    }
  ],
  "outputFieldMappings": [],
  "parameters": {
    "maxFailedItems": -1,
    "maxFailedItemsPerBatch": -1,
    "configuration": {
      "dataToExtract": "contentAndMetadata",
      "parsingMode": "default",
      "imageAction": "generateNormalizedImages"
    }
  }
}


r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


### Please wait for a few minutes after running the cell above to allow the Indexer to fill the Index

## And now... let's search!

Let's start with a text search. Recall:
- The **value** key contains the ***sequence of results*** returned by the query
- The **@search.answers** key contain the the ***answers*** query results for the search operation. They include three pieces of information:
  1. **key**: the search document key
  2. **score**: the score associated to that semantic answer
  3. **text**: the so called ***captions*** that contain the most representative passages from the document relatively to the search query. They are often used as document summary. Captions are only returned for queries of type semantic.

In [8]:
# Three full text answers plus two semantic answers
results = 3
answers = 2
QUESTION = "What is a meaning function?"

import requests, json

# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params  = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']} # NEW VERSION

# search query payload
search_payload = {
  "search": QUESTION,
  "count": "true",
  "top": results,
  "select": "id, name, title, chunk, language",
  "queryType": "semantic",
  "semanticConfiguration": "my-semantic-config",
  "captions": "extractive",
  "answers": f"extractive|count-{answers}"
}

r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name + "/docs/search.post.search",
                     data=json.dumps(search_payload), headers=headers, params=params)

print(json.dumps(r.json(), indent=2))

{
  "@odata.context": "https://cog-search-y24bi577jszf6.search.windows.net/indexes('cogsrch-index-files-vector-onestep')/$metadata#docs(*)",
  "@odata.count": 313,
  "@search.answers": [
    {
      "key": "8cac12fadbc1_aHR0cHM6Ly9ibG9ic3RvcmFnZXkyNGJpNTc3anN6ZjYuYmxvYi5jb3JlLndpbmRvd3MubmV0L2FyeGl2Y3MvMDAwMS8wMDAxMDAydjEucGRm0_chunks_1",
      "text": "A meaning function is a (possibly partial) function that maps sentences (and their parts) into (a representation of) their mean- ings; typically, some set-theoretic objects like lists of features or functions.",
      "highlights": "A meaning function is<em> a (possibly partial) function that maps sentences (and their parts) into (a representation of) their mean- ings;</em> typically, some set-theoretic objects like lists of features or functions.",
      "score": 0.9970703125
    },
    {
      "key": "8cac12fadbc1_aHR0cHM6Ly9ibG9ic3RvcmFnZXkyNGJpNTc3anN6ZjYuYmxvYi5jb3JlLndpbmRvd3MubmV0L2FyeGl2Y3MvMDAwMS8wMDAxMDAydjEucGRm0_chunks_4",
 

### Now let's do a *vector* query. 
We can use two possible ***vectorQueries*** settings in the search payload:
1. [**"kind": "vector"**](https://learn.microsoft.com/en-us/rest/api/searchservice/documents/search-post?view=rest-searchservice-2023-11-01&tabs=HTTP#rawvectorquery), when a raw vector value is provided, such as an Azure OpenAI Embedding. In this case, of course, we need to **manually** convert and pass the query embedding.
2. [**"kind": "text"**](https://learn.microsoft.com/en-us/rest/api/searchservice/documents/search-post?view=rest-searchservice-2023-10-01-Preview&tabs=HTTP#rawvectorquery), which accepts a text that is **automatically** converted into an embedding thanks to the **vectorized** parameter associated to the "chunkVector" field in the indexer. **Currently, only REST API 2023-10-01-Preview supports this feature**.

In [9]:
# "kind": "vector"

from langchain.embeddings import AzureOpenAIEmbeddings
embedder = AzureOpenAIEmbeddings(model="text-embedding-ada-002", skip_empty=True)
VECTORIZED_QUESTION = embedder.embed_query(QUESTION)
results = 3
answers = 2

# search query payload
search_payload = {
  "count": "true",
  "select": "id, name, title, location, chunk",
  "top": results,
  "vectorQueries": [
    {
      "kind": "vector",
      "k": answers,
      "fields": "chunkVector",
      "vector": VECTORIZED_QUESTION
    }
  ]
}

r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name + "/docs/search.post.search",
                     data=json.dumps(search_payload), headers=headers, params=params)

print(json.dumps(r.json(), indent=2))

{
  "@odata.context": "https://cog-search-y24bi577jszf6.search.windows.net/indexes('cogsrch-index-files-vector-onestep')/$metadata#docs(*)",
  "@odata.count": 2,
  "value": [
    {
      "@search.score": 0.84926814,
      "id": "b21f0042cdca_aHR0cHM6Ly9ibG9ic3RvcmFnZXkyNGJpNTc3anN6ZjYuYmxvYi5jb3JlLndpbmRvd3MubmV0L2FyeGl2Y3MvMDAwMS8wMDAxMDA2djEucGRm0_chunks_3",
      "title": "Microsoft Word - LP99-sub.htm",
      "chunk": "requirement that the meaning of an expression be functionally determined by its structure and the \n\nmeanings of its constituents.4 Furthermore, if this pre-theoretical sense of synonymy is crisply \n\ndefined, we can expect a suitably defined compositional function \u00b5 to reflect it. \n\n \n\n4. BEYOND COMPOSITIONALITY: SYSTEMATIC RELATIONAL THEORIES OF     \n\nMEANING \n\n \n\n    The importance which has been assigned to compositionality in semantic theory can be \n\nattributed to the fact that, until recently, many (most?) semanticists have identified it dire

In [10]:
# "kind": "text" - requires 2023-10-01-Preview version of Azure AI HTTP REST API

results = 3
answers = 2

# search query payload
search_payload = {
  "count": "true",
  "select": "id, name, title, location, chunk",
  "top": results,
  "vectorQueries": [
    {
      "kind": "text",
      "k": answers,
      "fields": "chunkVector", 
      "text": QUESTION
    }
  ]
}

r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name + "/docs/search.post.search",
                     data=json.dumps(search_payload), headers=headers, params=params_old) # 2023-10-01-Preview version

print(json.dumps(r.json(), indent=2))

{
  "@odata.context": "https://cog-search-y24bi577jszf6.search.windows.net/indexes('cogsrch-index-files-vector-onestep')/$metadata#docs(*)",
  "@odata.count": 2,
  "value": [
    {
      "@search.score": 0.84926814,
      "id": "b21f0042cdca_aHR0cHM6Ly9ibG9ic3RvcmFnZXkyNGJpNTc3anN6ZjYuYmxvYi5jb3JlLndpbmRvd3MubmV0L2FyeGl2Y3MvMDAwMS8wMDAxMDA2djEucGRm0_chunks_3",
      "title": "Microsoft Word - LP99-sub.htm",
      "chunk": "requirement that the meaning of an expression be functionally determined by its structure and the \n\nmeanings of its constituents.4 Furthermore, if this pre-theoretical sense of synonymy is crisply \n\ndefined, we can expect a suitably defined compositional function \u00b5 to reflect it. \n\n \n\n4. BEYOND COMPOSITIONALITY: SYSTEMATIC RELATIONAL THEORIES OF     \n\nMEANING \n\n \n\n    The importance which has been assigned to compositionality in semantic theory can be \n\nattributed to the fact that, until recently, many (most?) semanticists have identified it dire

## ...et *voilà*, the two answers are 100% identical
You want to confirm the differences, go [here](https://www.diffchecker.com/text-compare/) ;-)

# BONUS cell: using Azure OpenAI Extensions to make a single OpenAI call that implicitly contacts Azure AI Search
Finally, we make a single call to OpenAI, passing all the necessary information to access to Azure Search (its endpoint, authentication method, index name…) in the payload to extract the top chunks from the vector index we built in Azure AI Search.

## At this point, the question arises: Who calls Who?
…and the answer is that **OpenAI calls Azure Search**, there’s no doubt about it as we’ve only invoked the Azure OpenAI endpoint. So here’s how to implement this second method, which is certainly more practical than the previous one, even if potentially less flexible and controllable.

In [11]:
results = 3
answers = 2

openaicall_headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_OPENAI_API_KEY']}
openaicall_params  = {'api-version': '2023-12-01-preview'}

search_payload = {
    "count": "true",
    "select": "id, name, title, location, chunk",
    "top": results,
    "messages": [
        {
            "role": "user",
            "content": QUESTION
        }
    ],
    "temperature": 0,
    "top_p": 1.0,    
    "max_tokens": 1000,
    "dataSources": [
        {            
            "type": "AzureCognitiveSearch",
            "parameters": {                
                "endpoint": os.environ['AZURE_SEARCH_ENDPOINT'],
                "key": os.environ['AZURE_SEARCH_KEY'],
                "indexName": index_name
            }
        }
    ]
}

r = requests.post(
    f"{os.environ['AZURE_OPENAI_ENDPOINT']}openai/deployments/{os.environ['GPT4_32K_DEPLOYMENT']}/extensions/chat/completions",
    data=json.dumps(search_payload), headers=openaicall_headers, params=openaicall_params)

print(json.dumps(r.json(), indent=2))

{
  "id": "55eb15d5-8169-4f72-8a8c-25114c5be1a1",
  "model": "gpt-4-32k",
  "created": 1703298017,
  "object": "extensions.chat.completion",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "A meaning function, denoted as \u00b5, is a concept used in semantics to assign meanings to sentences and their parts. It is a compositional semantics for a set if its domain is contained in the set, and it satisfies the postulate of compositionality. This means that for all s, t in its domain, \u00b5(s.t) = \u00b5(s) \u2295 \u00b5(t), where s and t are sentences or parts of sentences. The meaning function should be the shortest, in the sense of the Minimum Description Length principle, and it should be maximal, meaning there is no \u00b5' with a larger domain that satisfies the same conditions[doc2]. \n\nIn the context of a language, a meaning function applies to expressions with fully specified syntactic repre